# データの性質を把握するために可視化

In [2]:
#ライブラリのインポート
import plotly.graph_objects as go
import polars as pl
import pandas as pd
import numpy as np
import pandas_ta as ta
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

In [3]:
#データの取得,ここではyahoo financeのapiを使用し、株価を取得
#データフレームに変換
y_share = share.Share('7203.T')
symbol_data = None
try:
    symbol_data = y_share.get_historical(
    share.PERIOD_TYPE_YEAR, 100,
    share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)
df = pd.DataFrame(symbol_data)
df["datetime"] = pd.to_datetime(df.timestamp, unit="ms")
df.head()

,timestamp,open,high,low,close,volume,datetime
0,925948800000,698.0,722.0,690.0,722.0,15575000,1999-05-06
1,926035200000,718.0,720.0,690.0,696.0,15165000,1999-05-07
2,926294400000,696.0,704.0,690.0,694.0,6305000,1999-05-10
3,926380800000,700.0,704.0,688.0,688.0,8430000,1999-05-11
4,926467200000,688.0,700.0,688.0,700.0,12980000,1999-05-12


## pandas_taは、テクニカル指標を計算するためのライブラリ
taを用いて、株価のリターンをあらかじめ対数変換や百分率変換した上で累積リターンを計算し、データフレームに追加している
理由は後述

In [4]:
# VWAP requires the DataFrame index to be a DatetimeIndex.
# Replace "datetime" with the appropriate column from your DataFrame
df.set_index(pd.DatetimeIndex(df["datetime"]), inplace=True)

# Calculate Returns and append to the df DataFrame
df.ta.log_return(cumulative=True, append=True)
df.ta.percent_return(cumulative=True, append=True)

# New Columns with results
df.columns

# Take a peek
df.tail()

,timestamp,open,high,low,close,volume,datetime,CUMLOGRET_1,CUMPCTRET_1
datetime,,,,,,,,,
2023-06-07 00:00:00,1686096000000,2058.0,2078.5,2021.0,2022.0,41062900,2023-06-07 00:00:00,1.029817,1.800554
2023-06-08 00:00:00,1686182400000,2038.5,2052.0,2017.5,2024.0,28562300,2023-06-08 00:00:00,1.030806,1.803324
2023-06-09 00:00:00,1686268800000,2055.0,2057.5,2030.0,2051.5,37276500,2023-06-09 00:00:00,1.044301,1.841413
2023-06-12 00:00:00,1686528000000,2065.5,2073.5,2054.5,2069.0,19714600,2023-06-12 00:00:00,1.052796,1.865651
2023-06-13 06:15:01,1686636901000,2100.0,2183.0,2099.5,2173.5,70125200,2023-06-13 06:15:01,1.102069,2.010388


## 折れ線グラフで終値をプロット

In [6]:
#plotly.graph_objectを使用してCUMLOGRETを折れ線で表示する
fig = go.Figure(data=go.Scatter(x=df.index, y=df['close']))
#グラフにタイトルを追加する
fig.update_layout(title='7203.T')
#x軸は年を表示する
fig.update_xaxes(
    tickformat="%Y",
    dtick="M12",
    ticklabelmode="period")
#グラフに軸ラベルを追加する
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Yen')
#グラフを表示する
fig.show()

#plotly.graph_objectを使用してCUMLOGRETを折れ線で表示する



時間経過による価格の変動を可視化した。
20年単位で見れば上昇トレンドにあることが視認できる。
上下動のノイズが大きいので、次は移動平均を利用してノイズを除去する。

In [16]:
#100日移動平均を計算する,append=Trueで元のデータフレームに追加,カラム名はSMA_100とつける
df.ta.sma(length=100, append=True, col="SMA_100")
#20日移動平均を計算する,append=Trueで元のデータフレームに追加,カラム名はSMA_20とつける
df.ta.sma(length=20, append=True, col="SMA_20")
#plotly.graph_objectを使用して100日移動平均を折れ線で表示する
fig = go.Figure(data=go.Scatter(x=df.index, y=df['SMA_100']))
#plotly.graph_objectを使用して20日移動平均を折れ線で表示する
fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20']))
#20,100日移動平均と一緒にplotly.graph_objectを使用してcloseを折れ線で表示する
fig.add_trace(go.Scatter(x=df.index, y=df['close']))
#trace 0に名前を付ける
fig.data[0].name = "SMA_100"
#trace 1に名前を付ける
fig.data[1].name = "SMA_20"
#trace 2に名前を付ける
fig.data[2].name = "close"

fig.update_layout(title='7203.T')
#x軸は年を表示する
fig.update_xaxes(
    tickformat="%Y",
    dtick="M12",
    ticklabelmode="period")
#グラフに軸ラベルを追加する
fig.update_xaxes(title_text='Year')
fig.update_yaxes(title_text='Yen')
#グラフを表示する
fig.show()


移動平均を利用することで、上下動のノイズを除去した。
過去のデータを利用するため、移動平均の値は過去にずれる。
100日移動平均を利用した場合,20日移動平均と比較して変化点が遅れることがわかる。

## apiを利用した株価データの取得では取引量も取得できるため、こちらも可視化を実施

In [47]:
#plotly_graph_objを使用してcloseを折れ線グラフで,volumeを棒グラフで表示する
trace1 = go.Scatter(x=df.index, y=df['close'], yaxis='y1',name='close')
#棒グラフを濃くする
trace2 = go.Bar(x=df.index, y=df['volume'],yaxis='y2',name='volume',opacity=0.5,marker=dict(color='black',
                line=dict(color='black',width=1)))
data = [trace1, trace2]
#第二軸にvolumeを表示する
layout = go.Layout(yaxis=dict(title='Yen', side='left'),
                   yaxis2=dict(title='',overlaying='y', side='right'), 
                    xaxis=dict( tickformat="%Y",
        dtick="M12",
        ticklabelmode="period")
        
        )
#グラフにタイトルを追加する
fig.update_layout(title='7203.T')
fig = go.Figure(data=data, layout=layout)
fig.show()

最も取引高が多かった日は、2010年5月6日であることがわかる。
取引高の多寡は、株価の変動に影響を与えると考えられるため、取引高の可視化も重要である。
実際にピンが立っている日は、取引高が多そうに見える。

## ヒストグラムで終値の分布をプロット
投資におけるKPIはリターンとまずは考えられるため、リターンの分布を確認する。

In [48]:
#前日とのcloseの差分によって、リターンを計算する
#ヒストグラムを作成し、リターンの分布を確認する
df['return'] = df['close'].diff()
fig = go.Figure(data=[go.Histogram(x=df['return'])])
fig.show()